In [3]:
import json
with open('youtube_credentials.json') as cred_data:
    info = json.load(cred_data)
    api_key = info["youtube_apikeyjson"]


from apiclient.discovery import build
youtube = build('youtube' , 'v3' , developerKey= api_key)


hello


# Get the videos for a particular keyword

In [3]:
# this simple function will get a list of videoids, from a particular given keyword. 

def getvideoids(stringquery, numresults = 25 ,
              beforepublished = None , afterpublished = None , rcode = None):
    
    from apiclient.discovery import build
    
    'this simple function will get a list of videoids, from a particular given string, arguments you can \
    specify are numresults, meaning the number of video extracted. Also beforepublished and afterpublishded, specifying \
    dates of interest for the extraction. Finaly, rcode stands for region code, for the exact formats you should check\
    the google data api for youtube python, as they might be subject to change. It corresponds to the publishedAfter,\
    publishedBefore and regioncode of the search() objects in the google api , here a link \
    https://developers.google.com/youtube/v3/docs/search/list '
    
    
    # Here we create a list where we will place the list of videos, we create an object that connects with the api.
    videolist = []
    youtube = build('youtube' , 'v3' , developerKey= api_key)
    
    # we make a call to the youtube object , asking for the list of videos. # Result orderds execution of the call
    funcall = youtube.search().list( q = stringquery ,part = "id", type = 'video', maxResults = numresults , 
                             publishedBefore = beforepublished , publishedAfter = afterpublished, regionCode = rcode ) 
    result = funcall.execute()
    
    
    # we use a for loop to place each of the different video results ids ( the only property we want) in the videolist

    for videoitem in range(0,len(result["items"])):
        videoitemid = result["items"][videoitem]["id"]["videoId"]
        videolist.append(videoitemid)
    return(videolist)

getvideoids("potato")


['UHKS37Inpdc',
 'l2j3-6_hsag',
 'Tq1KlMbBXQY',
 '93M1QtYDtpU',
 '-0tZHfRA7tg',
 'YWTGTWnxR2k',
 'zYNsQ6ibdZs',
 'EEHlCzBrBC0',
 '6XQo6RNpEdI',
 '3kRff5Zs9iI',
 'u8bT7BEaqaE',
 'a3pWBAbcPbg',
 'q7uyKYeGPdE',
 'rOuk5EHfBgw',
 'zcXREpN494A',
 'yyZfIivGJzs',
 'BAvFPTy4Rf4',
 'bNVxKaPxr6w',
 'fAnmSgDCx6I',
 '0AqJN9DTopQ',
 'tgfTnnWyhBA',
 'Qxkf9JNuK0g',
 'ICCqCTQ-lUU',
 '_wx__fEyDj0',
 'V5OpyhshHxA']

# Get the comment threads of a particular video id without replies 

In [5]:



def commentsearch(videoIdentification, partofcomment = "snippet", numbrofresult = 100):
    callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText")
    
    'this function on the other hand, obtains a list of comment threads for a given videoid.  Originally it will \
    retrieve 100 comments for each page ( one video can have multiple comment pages, that is the max allowed for the google api\
    here we will not retrieve comment replies ( the reason we dont by default, is because scraping replies slows down a LOT \
    the execution'

    results = callobject.execute()
    totallist = []
    
    # as before we created the call request object, and its execution where results will be stored. Here we create a 
    # nested function with the purpose of separating the parts of each of the results object that we want, ( they have a lot
    # of info, but we only want the important stuff) this separates it and creats a dictionary with the data, storing it
    # inside a list called totallist. 
    
    
    def incosearch() :
        for item in results["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"] ,
                     "publishingdate" : comment["snippet"]["publishedAt"] }
            totallist.append(resultdic)
            
    # we first call it once, to make sure to request the first comment page, te next piece of code uses the "nextPageTOken
    # property, making sure to keep calling for more and more commments, in case more comments exist. They are all stored 
    # in totallist. "
    incosearch()

        
    if ("nextPageToken" in results) == True : 
        while ("nextPageToken" in results) == True :
            callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText", pageToken = results["nextPageToken"])
            results = callobject.execute()
            
            incosearch()
            
        print(len(totallist))
                
        return(totallist)
    
    
    elif ("nextPageToken" in results) == False :
        print(len(totallist))
        return(totallist)

commentsearch('m14Dxbua9Vo')   



1113


[{'user': 'Laika',
  'usercomment': 'I mean there\'s a difference between "protagonist" and "hero" and the former isn\'t necessarily the latter.',
  'likecount': 0,
  'commentid': 'UgwITwtTPsAGGquO8ep4AaABAg',
  'publishingdate': '2019-01-09T19:10:15.000Z'},
 {'user': 'Red5',
  'usercomment': 'Not a singe mention of fortnite? You smash up buildings and make sniper towers for pete’s sake XD',
  'likecount': 0,
  'commentid': 'UgwaT4m_-tR9z0Le7ux4AaABAg',
  'publishingdate': '2019-01-08T04:22:42.000Z'},
 {'user': 'The Fanatic Mr.Falitico',
  'usercomment': "@outsidexbox While it wasn't major, you could always screw over Sven and or Faendal in Skyrim. When you meet them, they are in a dispute over Camilla. You can help Sven screw over Faendal, vise versa, or third option, take Camilla for yourself and screw over both. Personally, I help Faendal screw over Sven because you get Faendal as a follower.",
  'likecount': 0,
  'commentid': 'UgwkAEz7OVBqu8Acdnp4AaABAg',
  'publishingdate': '2019-

# Get the comment threads of a particular video id with replies

In [7]:

def commentsearchwithreply(videoIdentification, partofcomment = "snippet", numbrofresult = 100):
    
    callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText")
    import pandas as pd

    results = callobject.execute()
    totallist = []
    commentthreadlist = []
    
    ' this function extracts comments and its respective replies for a particular video, it works using the \
    getcommentresponses function inside '

    # here is the first major difference with the commentsearch function, we creadted another list
    # called the commentthreadlist where we will store the id of each comment obtained in a list, afterwards
    # we will use the getcommentresponses in a loop for  each of those comment ids in commentthreadlist
    
    def incosearch() :
        for item in results["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"] ,
                     "publishingdate" : comment["snippet"]["publishedAt"] }
            totallist.append(resultdic)
            commentthreadlist.append(resultdic["commentid"])
            
    # here we insert a function that will get comment replies for a particular comment thread, we will call it 
    # later in a loop
    
    def getcommentresponses(commentid, objectpart = "snippet" , resultnumber = 100 , textFormat = "plainText"  ):
        callobject = youtube.commentThreads().list(part = objectpart , maxResults = resultnumber, 
                                               textFormat = "plainText" , id = commentid)
        results = callobject.execute()
        for item in results["items"]:
            comment = item["snippet"]["topLevelComment"]
            author = comment["snippet"]["authorDisplayName"]
            text = comment["snippet"]["textDisplay"]
            resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"], 
                     "publishingdate" : comment["snippet"]["publishedAt"]}
            totallist.append(resultdic)
            
            
    incosearch() # this is a first call, without it, the first page will get skipped. 

        
    if ("nextPageToken" in results) == True : 
        while ("nextPageToken" in results) == True :
            callobject = youtube.commentThreads().list(part = partofcomment, videoId = videoIdentification, maxResults = numbrofresult, 
                                               textFormat = "plainText", pageToken = results["nextPageToken"])
            results = callobject.execute()
            
            incosearch() 
            
      
    
    elif ("nextPageToken" in results) == False :
        pass
    
    print("everything ok thus far")
    print(str(len(commentthreadlist)) + " are the number of comment threads, replies not included")
    
    # Here we get the comment replies 
    
    for i2 in commentthreadlist:
            getcommentresponses(i2)
    #here is we deliver the result of the function, its a list of dictionaries. 
    
    print(str(len(totallist)) + " are the number of comments extracted on this video, replies included.")
    return(totallist)

commentsearchwithreply('m14Dxbua9Vo')
   

everything ok thus far
1113 are the number of comment threads, replies not included
2226 are the number of comments extracted on this video, replies included.


[{'user': 'Laika',
  'usercomment': 'I mean there\'s a difference between "protagonist" and "hero" and the former isn\'t necessarily the latter.',
  'likecount': 0,
  'commentid': 'UgwITwtTPsAGGquO8ep4AaABAg',
  'publishingdate': '2019-01-09T19:10:15.000Z'},
 {'user': 'Red5',
  'usercomment': 'Not a singe mention of fortnite? You smash up buildings and make sniper towers for pete’s sake XD',
  'likecount': 0,
  'commentid': 'UgwaT4m_-tR9z0Le7ux4AaABAg',
  'publishingdate': '2019-01-08T04:22:42.000Z'},
 {'user': 'The Fanatic Mr.Falitico',
  'usercomment': "@outsidexbox While it wasn't major, you could always screw over Sven and or Faendal in Skyrim. When you meet them, they are in a dispute over Camilla. You can help Sven screw over Faendal, vise versa, or third option, take Camilla for yourself and screw over both. Personally, I help Faendal screw over Sven because you get Faendal as a follower.",
  'likecount': 0,
  'commentid': 'UgwkAEz7OVBqu8Acdnp4AaABAg',
  'publishingdate': '2019-

# Get comment replies for a particular comment id 


In [4]:
def getresponsesindependent(commentid, objectpart = "snippet" , resultnumber = 100 , textFormat = "plainText"  ):
    callobject = youtube.commentThreads().list(part = objectpart , maxResults = resultnumber, 
                                               textFormat = "plainText" , id = commentid)
    results = callobject.execute()
    'this function gets comment replies for a comment thread, a thread is equivalent to a comment obtained with \
    the comment search function, works just like getcommentresponses'
    
    totallist = []
    
    for item in results["items"]:
            
        comment = item["snippet"]["topLevelComment"]
        author = comment["snippet"]["authorDisplayName"]
        text = comment["snippet"]["textDisplay"]
        resultdic = {"user" : author ,"usercomment" : comment["snippet"]["textDisplay"], 
                   "likecount" : comment["snippet"]["likeCount"], "commentid" : item["id"], 
                     "publishingdate" : comment["snippet"]["publishedAt"]}
                
        totallist.append(resultdic)
                
    return(totallist)


getresponsesindependent('UgzNHNlVU6XjelpcJTp4AaABAg')
    
    
    

[{'user': 'LTxcheetosx21',
  'usercomment': 'Gordon Freeman and the microwace incident ?',
  'likecount': 0,
  'commentid': 'UgzNHNlVU6XjelpcJTp4AaABAg',
  'publishingdate': '2018-12-31T18:13:48.000Z'}]